In [299]:
%pylab inline
import numpy as np
import copy as cp
# %config Completer.use_jedi = False

Populating the interactive namespace from numpy and matplotlib


/opt/conda/envs/jupyter/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['gamma']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [300]:
def gamma_to_beta(gamma):
    return sqrt(1 - 1 / gamma**2)


def kinetic_to_betagamma(energy, rest_energy):
    gamma = energy / rest_energy + 1
    beta = gamma_to_beta(gamma)
    return beta, gamma

In [301]:
pcount = 3
m = 1.007319468  # amu
c = 299.792458  # mm/ns
q = 1.178734e-5  ##1.6E-19 C => 4.8032047E-10 statC [cm^(3/2)*g^(1/2)*s^(-1)] => [mm^(3/2)*amu^(1/2)*ns^(-1)];
aperture = 500  # mm
wall_pos = 30  # wall location on beam axis 'z', relative to origin


Px = np.random.normal(-3e-2, 3e-2, pcount)  # 3e-2 amu*mm/ns corresponds to 93 keV
Py = np.random.uniform(-3e-2, 3e-2, pcount)
Pz = np.random.uniform(
    1e6, 1e6, pcount
)  #  approx 1 TeV, CAREFUL WE ARE IN LAB FRAME HERE
Pt = np.sqrt(Px**2 + Py**2 + Pz**2 + m**2 * c**2)
gamma = Pt / (m * c)
bx = Px / (gamma * m * c)
by = Py / (gamma * m * c)
bz = Pz / (gamma * m * c)
beta_avg = np.sqrt(bx**2 + by**2 + bz**2)

x = numpy.random.uniform(-50, 50, pcount)
y = numpy.random.uniform(-50, 50, pcount)
z = numpy.random.uniform(-500, -450, pcount)
t = numpy.random.normal(
    0, 100 / (beta_avg * c), pcount
)  # from our old paper, not sure if this is necessary
# t = numpy.zeros(pcount)

bdotx = bx * np.random.uniform(
    -8e-1, 8e-2
)  # just a guess, there is probably much more acceleration going on in the core
bdoty = by * np.random.uniform(-8e-2, 8e-2)
bdotz = bz * np.random.uniform(
    -8e-2, 8e-2
)  # sloppy approximation to reflect much less deviation in logitudinal direction

In [302]:
bz

array([0.99999995, 0.99999995, 0.99999995])

In [303]:
beta_avg * c

array([299.79244433, 299.79244433, 299.79244433])

In [304]:
init_beam = {
    "x": x,
    "y": y,
    "z": z,
    "t": t,
    "Px": Px,
    "Py": Py,
    "Pz": Pz,
    "Pt": Pt,
    "bx": bx,
    "by": by,
    "bz": bz,
    "bdotx": bdotx,
    "bdoty": bdoty,
    "bdotz": bdotz,
    "gamma": gamma,
    "q": q,
}

In [305]:
bunch_rl = cp.deepcopy(init_beam)

In [306]:
init_beam["x"]

array([ 23.54038863,   6.89332564, -40.6129036 ])

In [307]:
init_beam["z"]

array([-457.52169361, -483.90274412, -498.71501901])

In [308]:
wallcount = pcount
q_w = -q  # one-to-one correlation for flat wall
Px_w = np.zeros(wallcount)
Py_w = np.zeros(wallcount)
Pz_w = np.zeros(wallcount)
Pt_w = np.zeros(wallcount)
gamma_w = np.zeros(wallcount)
bx_w = np.zeros(wallcount)
by_w = np.zeros(wallcount)
bz_w = np.zeros(wallcount)
beta_avg_w = np.sqrt(bx**2 + by**2 + bz**2)

x_w = np.zeros(wallcount)
y_w = np.zeros(wallcount)
z_w = np.zeros(wallcount)
t_w = np.zeros(
    wallcount
)  # ????? we need to initialize to some common timestep, I think this is consistent (we operate completely in the lab frame throughout)

bdotx_w = np.zeros(
    wallcount
)  # just a guess, there is probably much more acceleration going on in the core
bdoty_w = np.zeros(wallcount)
bdotz_w = np.zeros(wallcount)

In [309]:
# init_wall = {'x':x_w, 'y':y_w, 'z':z_w, 't':t_w, 'Px':Px_w, 'Py':Py_w, 'Pz':Pz_w,'Pt':Pt_w,
#             'bx':bx_w,'by':by_w,'bz':bz_w,'bdotx':bdotx_w,'bdoty':bdoty_w,'bdotz':bdotz_w,'gamma':gamma_w,'q':q_w}

In [310]:
def bouncer_flat(vector, wall_Z, apt_R):
    """
    taking losses at the circular aperture and generating a full image bunch reflecting off a flat wall

    includes cutoffs for particles striking wall or passing through the aperture
    """
    result = {}
    result["x"] = np.zeros_like(vector["x"])
    result["y"] = np.zeros_like(vector["y"])
    result["z"] = np.zeros_like(vector["z"])
    result["t"] = np.zeros_like(vector["t"])
    result["Px"] = np.zeros_like(vector["Px"])
    result["Py"] = np.zeros_like(vector["Py"])
    result["Pz"] = np.zeros_like(vector["Pz"])
    result["Pt"] = np.zeros_like(vector["Pt"])
    result["gamma"] = np.zeros_like(vector["gamma"])
    result["bx"] = np.zeros_like(vector["bx"])
    result["by"] = np.zeros_like(vector["by"])
    result["bz"] = np.zeros_like(vector["bz"])
    result["bdotx"] = np.zeros_like(vector["bdotx"])
    result["bdoty"] = np.zeros_like(vector["bdoty"])
    result["bdotz"] = np.zeros_like(vector["bdotz"])
    result["q"] = vector["q"]
    for i in range(len(vector["x"])):
        r = np.sqrt(vector["x"][i] ** 2 + vector["y"][i] ** 2)
        # sloppy way of taking losses, setting charge to zero
        # if vector['z'][i]>=wall_Z and r>=apt_R:
        #    vector['q'][i]=NaN #

        # vector['z'][i]<wall_Z and r<=apt_R:
        result["q"] = -vector["q"]
        result["z"][i] = wall_Z + 2 * (wall_Z - vector["z"][i])
        result["x"][i] = vector["x"][i]
        result["y"][i] = vector["y"][i]
        result["Px"][i] = vector["Px"][i]
        result["Py"][i] = vector["Py"][i]
        result["Pz"][i] = -vector["Pz"][i]
        result["Pt"][i] = vector["Pt"][i]  # right?
        result["gamma"][i] = vector["gamma"][i]
        result["bx"][i] = vector["bx"][i]
        result["by"][i] = vector["by"][i]
        result["bz"][i] = -vector["bz"][i]
        result["bdotx"][i] = vector["bdotx"][i]
        result["bdoty"][i] = vector["bdoty"][i]
        result["bdotz"][i] = -vector["bdotz"][i]
        result["t"][i] = vector["t"][
            i
        ]  # do NOT retard here, image charge is made to exist at the moment the original charge is created

    return result

In [311]:
init_beam["z"]

array([-457.52169361, -483.90274412, -498.71501901])

In [312]:
bunch_img = bouncer_flat(init_beam, wall_pos, aperture)
# external_particle = ?????

In [313]:
init_beam["z"]

array([-457.52169361, -483.90274412, -498.71501901])

In [314]:
bunch_rl["z"]

array([-457.52169361, -483.90274412, -498.71501901])

In [315]:
bunch_img["t"]

array([ 0.4751472 , -0.02042117, -0.30191797])

In [316]:
# def chrono_matcher(vector_set,vector_ext_set,index_global_t,index_particle):
#     """
#     scans back though external particle histories to find matching time point for sample particle and

#     Times are all in lab-frame. vector_set and vector_ext_set are full trajectories. index_global_t is the current time step
#     ONLY WORKS FOR ONE SAMPLE PARTICLE AT A TIME -- But generates matched times for all particles in the corresponding image bunch
#     """
#     vector_ext_set_matched = copy(vector_ext_set)
#     if index==0:
#         raise Exception('vector_ext_set must have a history -- cannot begin at t=0')

#     #rudimentary scan, should be as close to zero as possible, but just overshooting by a time step should be conservative enough for now.
#     for j in range(len(vector_ext_set[index]['t'])):
#             k = 1
#             while vector_set[index_global_t]['t'][index_particle]-vector_ext_set[index_global_t-k]['t'][j] > 0.1:
#                 k += 1
#             else:
#                 vector_ext_set_matched[index]['t'][j] = vector_ext_set[index-k]['t'][j]

# return(vector_ext_set_matched)


In [317]:
def dist_euclid(vector, vector_ext, index):
    """
    simple Euclidean distance generator with our standard retarded-potential notation

    requires chrono_match-ed trajectories
    """
    result = {}
    result["R"] = np.zeros_like(vector["x"])
    result["nx"] = np.zeros_like(vector["x"])
    result["ny"] = np.zeros_like(vector["x"])
    result["nz"] = np.zeros_like(vector["x"])
    for j in range(len(vector_ext["x"])):
        result["R"][j] = np.sqrt(
            (vector["x"][index] - vector_ext["x"][j]) ** 2
            + (vector["y"][index] - vector_ext["y"][j]) ** 2
            + (vector["z"][index] - vector_ext["z"][j]) ** 2
        )
        result["nx"][j] = (vector["x"][index] - vector_ext["x"][j]) / result["R"][j]
        result["ny"][j] = (vector["y"][index] - vector_ext["y"][j]) / result["R"][j]
        result["nz"][j] = (vector["z"][index] - vector_ext["z"][j]) / result["R"][j]
    return result

In [318]:
def eqsofmotion_static(
    h, vector, vector_ext
):  # nhat includes R and fnhat components, need to generate this per particle pair
    result = {}
    result["x"] = np.zeros_like(vector["x"])
    result["y"] = np.zeros_like(vector["y"])
    result["z"] = np.zeros_like(vector["z"])
    result["t"] = np.zeros_like(vector["t"])
    result["Px"] = np.zeros_like(vector["Px"])
    result["Py"] = np.zeros_like(vector["Py"])
    result["Pz"] = np.zeros_like(vector["Pz"])
    result["Pt"] = np.zeros_like(vector["Pt"])
    result["gamma"] = np.zeros_like(vector["gamma"])
    result["bx"] = np.zeros_like(vector["bx"])
    result["by"] = np.zeros_like(vector["by"])
    result["bz"] = np.zeros_like(vector["bz"])
    result["bdotx"] = np.zeros_like(vector["bdotx"])
    result["bdoty"] = np.zeros_like(vector["bdoty"])
    result["bdotz"] = np.zeros_like(vector["bdotz"])
    result["q"] = vector["q"]
    for i in range(
        len(vector["x"])
    ):  # iterating over all real particles OR all reflection points (these must be done in separate steps)
        nhat = dist_euclid(vector, vector_ext, i)
        for j in range(
            len(vector_ext["x"])
        ):  # summing all external contributions (reflected particles and/or local particles)
            # b_nhat = vector['bx'][i]*nhat['nx'][j]+vector['by'][i]*nhat['ny'][j]+vector['bz'][i]*nhat['nz'][j] #for accurate chrono-matching
            result["x"][i] = vector["x"][i]
            result["y"][i] = vector["y"][i]
            result["z"][i] = vector["z"][i]
            result["t"][i] = vector["t"][i]
            result["Px"][i] = vector["Px"][i] + h / m * vector["q"] * vector_ext[
                "q"
            ] * vector["gamma"][i] * 1 / (
                1
                - np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                )
                ** 2
            ) * (
                1
                / nhat["R"][j] ** 2
                * vector["bx"][i]
                * np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                )
                - nhat["nx"][j]
                * np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                )
                / (
                    vector_ext["gamma"][j] ** 2
                    * nhat["R"][j] ** 2
                    * (
                        1
                        - np.dot(
                            (
                                vector_ext["bx"][j],
                                vector_ext["by"][j],
                                vector_ext["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
                - nhat["nx"][j]
                / (c * nhat["R"][j])
                * (
                    np.dot(
                        (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                        (
                            (
                                vector_ext["bdotx"][j],
                                vector_ext["bdoty"][j],
                                vector_ext["bdotz"][j],
                            )
                            + vector_ext["gamma"][j] ** 2
                            * np.dot(
                                (
                                    vector_ext["bx"][j],
                                    vector_ext["by"][j],
                                    vector_ext["bz"][j],
                                ),
                                np.dot(
                                    (
                                        vector_ext["bx"][j],
                                        vector_ext["by"][j],
                                        vector_ext["bz"][j],
                                    ),
                                    (
                                        vector_ext["bdotx"][j],
                                        vector_ext["bdoty"][j],
                                        vector_ext["bdotz"][j],
                                    ),
                                ),
                            )
                        ),
                    )
                )
            )
            result["Py"][i] = vector["Py"][i] + h / m * vector["q"] * vector_ext[
                "q"
            ] * vector["gamma"][i] * 1 / (
                1
                - np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                )
                ** 2
            ) * (
                1
                / nhat["R"][j] ** 2
                * vector["by"][i]
                * np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                )
                - nhat["ny"][j]
                * np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                )
                / (
                    vector_ext["gamma"][j] ** 2
                    * nhat["R"][j] ** 2
                    * (
                        1
                        - np.dot(
                            (
                                vector_ext["bx"][j],
                                vector_ext["by"][j],
                                vector_ext["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
                - nhat["ny"][j]
                / (c * nhat["R"][j])
                * (
                    np.dot(
                        (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                        (
                            (
                                vector_ext["bdotx"][j],
                                vector_ext["bdoty"][j],
                                vector_ext["bdotz"][j],
                            )
                            + vector_ext["gamma"][j] ** 2
                            * np.dot(
                                (
                                    vector_ext["bx"][j],
                                    vector_ext["by"][j],
                                    vector_ext["bz"][j],
                                ),
                                np.dot(
                                    (
                                        vector_ext["bx"][j],
                                        vector_ext["by"][j],
                                        vector_ext["bz"][j],
                                    ),
                                    (
                                        vector_ext["bdotx"][j],
                                        vector_ext["bdoty"][j],
                                        vector_ext["bdotz"][j],
                                    ),
                                ),
                            )
                        ),
                    )
                )
            )
            result["Pz"][i] = vector["Pz"][i] + h / m * vector["q"] * vector_ext[
                "q"
            ] * vector["gamma"][i] * 1 / (
                1
                - np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                )
                ** 2
            ) * (
                1
                / nhat["R"][j] ** 2
                * vector["bz"][i]
                * np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                )
                - nhat["nz"][j]
                * np.dot(
                    (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                    (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                )
                / (
                    vector_ext["gamma"][j] ** 2
                    * nhat["R"][j] ** 2
                    * (
                        1
                        - np.dot(
                            (
                                vector_ext["bx"][j],
                                vector_ext["by"][j],
                                vector_ext["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
                - nhat["nz"][j]
                / (c * nhat["R"][j])
                * (
                    np.dot(
                        (vector["bx"][i], vector["by"][i], vector["bz"][i]),
                        (
                            (
                                vector_ext["bdotx"][j],
                                vector_ext["bdoty"][j],
                                vector_ext["bdotz"][j],
                            )
                            + vector_ext["gamma"][j] ** 2
                            * np.dot(
                                (
                                    vector_ext["bx"][j],
                                    vector_ext["by"][j],
                                    vector_ext["bz"][j],
                                ),
                                np.dot(
                                    (
                                        vector_ext["bx"][j],
                                        vector_ext["by"][j],
                                        vector_ext["bz"][j],
                                    ),
                                    (
                                        vector_ext["bdotx"][j],
                                        vector_ext["bdoty"][j],
                                        vector_ext["bdotz"][j],
                                    ),
                                ),
                            )
                        ),
                    )
                )
            )
            result["Pt"][i] = vector["Pt"][i] + h / m * vector["q"] * vector_ext[
                "q"
            ] * vector["gamma"][i] * (
                (
                    (
                        vector_ext["gamma"][j] ** 2
                        * np.dot(
                            (
                                vector_ext["bx"][j],
                                vector_ext["by"][j],
                                vector_ext["bz"][j],
                            ),
                            (
                                vector_ext["bdotx"][j],
                                vector_ext["bdoty"][j],
                                vector_ext["bdotz"][j],
                            ),
                        )
                    )
                    / (c * nhat["R"][j])
                    - 1 / nhat["R"][j] ** 2
                )
                / (
                    1
                    - np.dot(
                        (vector_ext["bx"][j], vector_ext["by"][j], vector_ext["bz"][j]),
                        (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                    )
                )
            )
            result["gamma"][i] = result["Pt"][i] / (m * c)
            result["bx"][i] = result["Px"][i] / (m * c * result["gamma"][i])
            result["by"][i] = result["Py"][i] / (m * c * result["gamma"][i])
            result["bz"][i] = result["Pz"][i] / (m * c * result["gamma"][i])
            result["bdotx"][i] = (
                (result["Px"][i] - vector["Px"][i]) / h
            )  # necessary history to treat as an external particle later (keep in lab frame?)
            result["bdoty"][i] = (result["Py"][i] - vector["Py"][i]) / h
            result["bdotz"][i] = (result["Pz"][i] - vector["Pz"][i]) / h

            # NOTE---- Momentum values below are updated 'result', implicit technically, but only needs explicit solver for extreme
            result["x"][i] = vector["x"][i] + h / m * (
                result["Px"][i]
                + vector["q"]
                / c
                * vector_ext["q"]
                * vector_ext["bx"][j]
                / (
                    vector_ext["gamma"][j]
                    * c
                    * nhat["R"][j]
                    * (
                        1
                        - np.dot(
                            (
                                vector_ext["bx"][j],
                                vector_ext["by"][j],
                                vector_ext["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
            )
            result["y"][i] = vector["y"][i] + h / m * (
                result["Py"][i]
                + vector["q"]
                / c
                * vector_ext["q"]
                * vector_ext["by"][j]
                / (
                    vector_ext["gamma"][j]
                    * c
                    * nhat["R"][j]
                    * (
                        1
                        - np.dot(
                            (
                                vector_ext["bx"][j],
                                vector_ext["by"][j],
                                vector_ext["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
            )
            result["z"][i] = vector["z"][i] + h / m * (
                result["Pz"][i]
                + vector["q"]
                / c
                * vector_ext["q"]
                * vector_ext["bz"][j]
                / (
                    vector_ext["gamma"][j]
                    * c
                    * nhat["R"][j]
                    * (
                        1
                        - np.dot(
                            (
                                vector_ext["bx"][j],
                                vector_ext["by"][j],
                                vector_ext["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
            )
            result["t"][i] = vector["t"][i] + h / m * (
                result["Pt"][i]
                + vector["q"]
                / c
                * vector_ext["q"]
                / (
                    vector_ext["gamma"][j]
                    * c
                    * nhat["R"][j]
                    * (
                        1
                        - np.dot(
                            (
                                vector_ext["bx"][j],
                                vector_ext["by"][j],
                                vector_ext["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
            )
    return result

In [319]:
def static_integrator(steps, h_step, bunch_real, wall_Z, apt_R):
    trajectory = [{}] * steps
    trajectory_img = [{}] * steps
    for i in range(steps):
        if i == 0:
            trajectory[i] = init_beam
            trajectory_img[i] = bouncer_flat(
                init_beam, wall_Z, apt_R
            )  # note that init_wall is a dummy vector
        else:
            trajectory[i] = eqsofmotion_static(
                h_step, trajectory[i - 1], trajectory_img[i - 1]
            )
            trajectory_img[i] = bouncer_flat(
                trajectory[i], wall_Z, apt_R
            )  # note that init_wall is a dummy vector
    return trajectory, trajectory_img

In [320]:
thermalized_bunch, thermalized_img = static_integrator(
    300, 1.3e-6, init_beam, wall_pos, aperture
)

In [321]:
# def chron_matcher(traj_rl,traj_img,nhat):

In [322]:
init_beam

{'x': array([ 23.54038863,   6.89332564, -40.6129036 ]),
 'y': array([ 27.71666746, -48.03611048,  30.66345474]),
 'z': array([-457.52169361, -483.90274412, -498.71501901]),
 't': array([ 0.4751472 , -0.02042117, -0.30191797]),
 'Px': array([-0.05148072, -0.05498809,  0.01412652]),
 'Py': array([ 0.01174733, -0.02681457,  0.02746711]),
 'Pz': array([1000000., 1000000., 1000000.]),
 'Pt': array([1000000.04559801, 1000000.04559801, 1000000.04559801]),
 'bx': array([-5.14807175e-08, -5.49880863e-08,  1.41265162e-08]),
 'by': array([ 1.17473264e-08, -2.68145676e-08,  2.74671075e-08]),
 'bz': array([0.99999995, 0.99999995, 0.99999995]),
 'bdotx': array([ 6.29667162e-09,  6.72566233e-09, -1.72783205e-09]),
 'bdoty': array([-8.21552022e-10,  1.87528307e-09, -1.92091859e-09]),
 'bdotz': array([-0.06058842, -0.06058842, -0.06058842]),
 'gamma': array([3311.40339291, 3311.40339291, 3311.40339291]),
 'q': 1.178734e-05}

In [323]:
thermalized_bunch[299]

{'x': array([ 23.54036923,   6.89330459, -40.61289815]),
 'y': array([ 27.71667198, -48.03612111,  30.66346533]),
 'z': array([ -71.64610878,  -98.02715373, -112.84124308]),
 't': array([386.35078291, 385.85520343, 385.57733556]),
 'Px': array([-0.05025043, -0.05454121,  0.01412652]),
 'Py': array([ 0.01169082, -0.02755488,  0.02746711]),
 'Pz': array([999999.97075364, 999999.98540612, 999995.21717976]),
 'Pt': array([1000000.10411691, 1000000.07481149, 1000009.61131415]),
 'gamma': array([3311.40358669, 3311.40348965, 3311.43506885]),
 'bx': array([-5.02504282e-08, -5.45412049e-08,  1.41263811e-08]),
 'by': array([ 1.16908143e-08, -2.75548749e-08,  2.74668447e-08]),
 'bz': array([0.99999987, 0.99999991, 0.99998561]),
 'bdotx': array([0., 0., 0.]),
 'bdoty': array([0., 0., 0.]),
 'bdotz': array([0., 0., 0.]),
 'q': 1.178734e-05}

In [324]:
thermalized_img[299]

{'x': array([ 23.54036923,   6.89330459, -40.61289815]),
 'y': array([ 27.71667198, -48.03612111,  30.66346533]),
 'z': array([233.29221755, 286.05430747, 315.68248616]),
 't': array([386.35078291, 385.85520343, 385.57733556]),
 'Px': array([-0.05025043, -0.05454121,  0.01412652]),
 'Py': array([ 0.01169082, -0.02755488,  0.02746711]),
 'Pz': array([-999999.97075364, -999999.98540612, -999995.21717976]),
 'Pt': array([1000000.10411691, 1000000.07481149, 1000009.61131415]),
 'gamma': array([3311.40358669, 3311.40348965, 3311.43506885]),
 'bx': array([-5.02504282e-08, -5.45412049e-08,  1.41263811e-08]),
 'by': array([ 1.16908143e-08, -2.75548749e-08,  2.74668447e-08]),
 'bz': array([-0.99999987, -0.99999991, -0.99998561]),
 'bdotx': array([0., 0., 0.]),
 'bdoty': array([0., 0., 0.]),
 'bdotz': array([-0., -0., -0.]),
 'q': -1.178734e-05}

In [332]:
thermalized_bunch[1]["t"]

array([1.76570109, 1.27013272, 0.98863592])

### delta_t = R/c*(1+beta_n)

In [333]:
def chrono_jn(trajectory, trajectory_ext):
    for i in range(len(trajectory)):
        for l in range(len(trajectory[i]["x"])):
            nhat = dist_euclid(trajectory[i], trajectory_ext[i], l)
            b_nhat = (
                trajectory[i]["bx"][l] * nhat["nx"][j]
                + trajectory[i]["by"][l] * nhat["ny"][j]
                + trajectory[i]["bz"][l] * nhat["nz"][j]
            )  # for accurate chrono-matching
            # b_nhat = trajectory[i]['bz'][l]*nhat['nz'][j] #for speedup

            delta_t = nhat["R"] * (1 + b_nhat)
            t_ext_new = trajectory[i]["t"][l] - delta_t

            ####NOW MATCH
            index_new = []

    return (nhat, index_new)

In [326]:
def eqsofmotion_retarded(
    h, trajectory, trajectory_ext
):  # nhat includes R and fnhat components, need to generate this per particle pair
    result = [{}]
    for i in range(len(trajectory)):
        result[i]["x"] = np.zeros_like(trajectory[i]["x"])
        result[i]["y"] = np.zeros_like(trajectory[i]["y"])
        result[i]["z"] = np.zeros_like(trajectory[i]["z"])
        result[i]["t"] = np.zeros_like(trajectory[i]["t"])
        result[i]["Px"] = np.zeros_like(trajectory[i]["Px"])
        result[i]["Py"] = np.zeros_like(trajectory[i]["Py"])
        result[i]["Pz"] = np.zeros_like(trajectory[i]["Pz"])
        result[i]["Pt"] = np.zeros_like(trajectory[i]["Pt"])
        result[i]["gamma"] = np.zeros_like(trajectory[i]["gamma"])
        result[i]["bx"] = np.zeros_like(trajectory[i]["bx"])
        result[i]["by"] = np.zeros_like(trajectory[i]["by"])
        result[i]["bz"] = np.zeros_like(trajectory[i]["bz"])
        result[i]["bdotx"] = np.zeros_like(trajectory[i]["bdotx"])
        result[i]["bdoty"] = np.zeros_like(trajectory[i]["bdoty"])
        result[i]["bdotz"] = np.zeros_like(trajectory[i]["bdotz"])
        result[i]["q"] = trajectory[i]["q"]
    for l in range(
        len(trajectory[i]["x"])
    ):  # iterating over all real particles OR all reflection points (these must be done in separate steps)
        nhat = dist_euclid(vector, vector_ext, l)
        for j in range(
            len(trajectory_ext[i]["x"])
        ):  # summing all external contributions (reflected particles and/or local particles)
            # b_nhat = trajectory[i]['bx'][l]*nhat['nx'][j]+trajectory[i]['by'][l]*nhat['ny'][j]+trajectory[i]['bz'][l]*nhat['nz'][j] #for accurate chrono-matching
            result[i]["x"][l] = trajectory[i]["x"][l]
            result[i]["y"][l] = trajectory[i]["y"][l]
            result[i]["z"][l] = trajectory[i]["z"][l]
            result[i]["t"][l] = trajectory[i]["t"][l]
            result[i]["Px"][l] = trajectory[i]["Px"][l] + h / m * trajectory[i][
                "q"
            ] * trajectory_ext[i]["q"] * trajectory[i]["gamma"][l] * 1 / (
                1
                - np.dot(
                    (
                        trajectory_ext[i]["bx"][j],
                        trajectory_ext[i]["by"][j],
                        trajectory_ext[i]["bz"][j],
                    ),
                    (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                )
                ** 2
            ) * (
                1
                / nhat["R"][j] ** 2
                * trajectory[i]["bx"][l]
                * np.dot(
                    (
                        trajectory_ext[i]["bx"][j],
                        trajectory_ext[i]["by"][j],
                        trajectory_ext[i]["bz"][j],
                    ),
                    (
                        trajectory[i]["bx"][l],
                        trajectory[i]["by"][l],
                        trajectory[i]["bz"][l],
                    ),
                )
                - nhat["nx"][j]
                * np.dot(
                    (
                        trajectory_ext[i]["bx"][j],
                        trajectory_ext[i]["by"][j],
                        trajectory_ext[i]["bz"][j],
                    ),
                    (
                        trajectory[i]["bx"][l],
                        trajectory[i]["by"][l],
                        trajectory[i]["bz"][l],
                    ),
                )
                / (
                    trajectory_ext[i]["gamma"][j] ** 2
                    * nhat["R"][j] ** 2
                    * (
                        1
                        - np.dot(
                            (
                                trajectory_ext[i]["bx"][j],
                                trajectory_ext[i]["by"][j],
                                trajectory_ext[i]["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
                - nhat["nx"][j]
                / (c * nhat["R"][j])
                * (
                    np.dot(
                        (
                            trajectory[i]["bx"][l],
                            trajectory[i]["by"][l],
                            trajectory[i]["bz"][l],
                        ),
                        (
                            (
                                trajectory_ext[i]["bdotx"][j],
                                trajectory_ext[i]["bdoty"][j],
                                trajectory_ext[i]["bdotz"][j],
                            )
                            + trajectory_ext[i]["gamma"][j] ** 2
                            * np.dot(
                                (
                                    trajectory_ext[i]["bx"][j],
                                    trajectory_ext[i]["by"][j],
                                    trajectory_ext[i]["bz"][j],
                                ),
                                np.dot(
                                    (
                                        trajectory_ext[i]["bx"][j],
                                        trajectory_ext[i]["by"][j],
                                        trajectory_ext[i]["bz"][j],
                                    ),
                                    (
                                        trajectory_ext[i]["bdotx"][j],
                                        trajectory_ext[i]["bdoty"][j],
                                        trajectory_ext[i]["bdotz"][j],
                                    ),
                                ),
                            )
                        ),
                    )
                )
            )
            result[i]["Py"][l] = trajectory[i]["Py"][l] + h / m * trajectory[i][
                "q"
            ] * trajectory_ext[i]["q"] * trajectory[i]["gamma"][l] * 1 / (
                1
                - np.dot(
                    (
                        trajectory_ext[i]["bx"][j],
                        trajectory_ext[i]["by"][j],
                        trajectory_ext[i]["bz"][j],
                    ),
                    (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                )
                ** 2
            ) * (
                1
                / nhat["R"][j] ** 2
                * trajectory[i]["by"][l]
                * np.dot(
                    (
                        trajectory_ext[i]["bx"][j],
                        trajectory_ext[i]["by"][j],
                        trajectory_ext[i]["bz"][j],
                    ),
                    (
                        trajectory[i]["bx"][l],
                        trajectory[i]["by"][l],
                        trajectory[i]["bz"][l],
                    ),
                )
                - nhat["ny"][j]
                * np.dot(
                    (
                        trajectory_ext[i]["bx"][j],
                        trajectory_ext[i]["by"][j],
                        trajectory_ext[i]["bz"][j],
                    ),
                    (
                        trajectory[i]["bx"][l],
                        trajectory[i]["by"][l],
                        trajectory[i]["bz"][l],
                    ),
                )
                / (
                    trajectory_ext[i]["gamma"][j] ** 2
                    * nhat["R"][j] ** 2
                    * (
                        1
                        - np.dot(
                            (
                                trajectory_ext[i]["bx"][j],
                                trajectory_ext[i]["by"][j],
                                trajectory_ext[i]["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
                - nhat["ny"][j]
                / (c * nhat["R"][j])
                * (
                    np.dot(
                        (
                            trajectory[i]["bx"][l],
                            trajectory[i]["by"][l],
                            trajectory[i]["bz"][l],
                        ),
                        (
                            (
                                trajectory_ext[i]["bdotx"][j],
                                trajectory_ext[i]["bdoty"][j],
                                trajectory_ext[i]["bdotz"][j],
                            )
                            + trajectory_ext[i]["gamma"][j] ** 2
                            * np.dot(
                                (
                                    trajectory_ext[i]["bx"][j],
                                    trajectory_ext[i]["by"][j],
                                    trajectory_ext[i]["bz"][j],
                                ),
                                np.dot(
                                    (
                                        trajectory_ext[i]["bx"][j],
                                        trajectory_ext[i]["by"][j],
                                        trajectory_ext[i]["bz"][j],
                                    ),
                                    (
                                        trajectory_ext[i]["bdotx"][j],
                                        trajectory_ext[i]["bdoty"][j],
                                        trajectory_ext[i]["bdotz"][j],
                                    ),
                                ),
                            )
                        ),
                    )
                )
            )
            result[i]["Pz"][l] = trajectory[i]["Pz"][l] + h / m * trajectory[i][
                "q"
            ] * trajectory_ext[i]["q"] * trajectory[i]["gamma"][l] * 1 / (
                1
                - np.dot(
                    (
                        trajectory_ext[i]["bx"][j],
                        trajectory_ext[i]["by"][j],
                        trajectory_ext[i]["bz"][j],
                    ),
                    (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                )
                ** 2
            ) * (
                1
                / nhat["R"][j] ** 2
                * trajectory[i]["bz"][l]
                * np.dot(
                    (
                        trajectory_ext[i]["bx"][j],
                        trajectory_ext[i]["by"][j],
                        trajectory_ext[i]["bz"][j],
                    ),
                    (
                        trajectory[i]["bx"][l],
                        trajectory[i]["by"][l],
                        trajectory[i]["bz"][l],
                    ),
                )
                - nhat["nz"][j]
                * np.dot(
                    (
                        trajectory_ext[i]["bx"][j],
                        trajectory_ext[i]["by"][j],
                        trajectory_ext[i]["bz"][j],
                    ),
                    (
                        trajectory[i]["bx"][l],
                        trajectory[i]["by"][l],
                        trajectory[i]["bz"][l],
                    ),
                )
                / (
                    trajectory_ext[i]["gamma"][j] ** 2
                    * nhat["R"][j] ** 2
                    * (
                        1
                        - np.dot(
                            (
                                trajectory_ext[i]["bx"][j],
                                trajectory_ext[i]["by"][j],
                                trajectory_ext[i]["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
                - nhat["nz"][j]
                / (c * nhat["R"][j])
                * (
                    np.dot(
                        (
                            trajectory[i]["bx"][l],
                            trajectory[i]["by"][l],
                            trajectory[i]["bz"][l],
                        ),
                        (
                            (
                                trajectory_ext[i]["bdotx"][j],
                                trajectory_ext[i]["bdoty"][j],
                                trajectory_ext[i]["bdotz"][j],
                            )
                            + trajectory_ext[i]["gamma"][j] ** 2
                            * np.dot(
                                (
                                    trajectory_ext[i]["bx"][j],
                                    trajectory_ext[i]["by"][j],
                                    trajectory_ext[i]["bz"][j],
                                ),
                                np.dot(
                                    (
                                        trajectory_ext[i]["bx"][j],
                                        trajectory_ext[i]["by"][j],
                                        trajectory_ext[i]["bz"][j],
                                    ),
                                    (
                                        trajectory_ext[i]["bdotx"][j],
                                        trajectory_ext[i]["bdoty"][j],
                                        trajectory_ext[i]["bdotz"][j],
                                    ),
                                ),
                            )
                        ),
                    )
                )
            )
            result[i]["Pt"][l] = trajectory[i]["Pt"][l] + h / m * trajectory[i][
                "q"
            ] * trajectory_ext[i]["q"] * trajectory[i]["gamma"][l] * (
                (
                    (
                        trajectory_ext[i]["gamma"][j] ** 2
                        * np.dot(
                            (
                                trajectory_ext[i]["bx"][j],
                                trajectory_ext[i]["by"][j],
                                trajectory_ext[i]["bz"][j],
                            ),
                            (
                                trajectory_ext[i]["bdotx"][j],
                                trajectory_ext[i]["bdoty"][j],
                                trajectory_ext[i]["bdotz"][j],
                            ),
                        )
                    )
                    / (c * nhat["R"][j])
                    - 1 / nhat["R"][j] ** 2
                )
                / (
                    1
                    - np.dot(
                        (
                            trajectory_ext[i]["bx"][j],
                            trajectory_ext[i]["by"][j],
                            trajectory_ext[i]["bz"][j],
                        ),
                        (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                    )
                )
            )
            result[i]["gamma"][l] = result[i]["Pt"][l] / (m * c)
            result[i]["bx"][l] = result[i]["Px"][l] / (m * c * result[i]["gamma"][l])
            result[i]["by"][l] = result[i]["Py"][l] / (m * c * result[i]["gamma"][l])
            result[i]["bz"][l] = result[i]["Pz"][l] / (m * c * result[i]["gamma"][l])
            result[i]["bdotx"][l] = (
                (result[i]["Px"][l] - trajectory[i]["Px"][l]) / h
            )  # necessary history to treat as an external particle later (keep in lab frame?)
            result[i]["bdoty"][l] = (result[i]["Py"][l] - trajectory[i]["Py"][l]) / h
            result[i]["bdotz"][l] = (result[i]["Pz"][l] - trajectory[i]["Pz"][l]) / h

            # NOTE---- Momentum values below are updated 'result', implicit technically, but only needs explicit solver for extreme
            result[i]["x"][l] = trajectory[i]["x"][l] + h / m * (
                result[i]["Px"][l]
                + trajectory[i]["q"]
                / c
                * trajectory_ext[i]["q"]
                * trajectory_ext[i]["bx"][j]
                / (
                    trajectory_ext[i]["gamma"][j]
                    * c
                    * nhat["R"][j]
                    * (
                        1
                        - np.dot(
                            (
                                trajectory_ext[i]["bx"][j],
                                trajectory_ext[i]["by"][j],
                                trajectory_ext[i]["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
            )
            result[i]["y"][l] = trajectory[i]["y"][l] + h / m * (
                result[i]["Py"][l]
                + trajectory[i]["q"]
                / c
                * trajectory_ext[i]["q"]
                * trajectory_ext[i]["by"][j]
                / (
                    trajectory_ext[i]["gamma"][j]
                    * c
                    * nhat["R"][j]
                    * (
                        1
                        - np.dot(
                            (
                                trajectory_ext[i]["bx"][j],
                                trajectory_ext[i]["by"][j],
                                trajectory_ext[i]["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
            )
            result[i]["z"][l] = trajectory[i]["z"][l] + h / m * (
                result[i]["Pz"][l]
                + trajectory[i]["q"]
                / c
                * trajectory_ext[i]["q"]
                * trajectory_ext[i]["bz"][j]
                / (
                    trajectory_ext[i]["gamma"][j]
                    * c
                    * nhat["R"][j]
                    * (
                        1
                        - np.dot(
                            (
                                trajectory_ext[i]["bx"][j],
                                trajectory_ext[i]["by"][j],
                                trajectory_ext[i]["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
            )
            result[i]["t"][l] = trajectory[i]["t"][l] + h / m * (
                result[i]["Pt"][l]
                + trajectory[i]["q"]
                / c
                * trajectory_ext[i]["q"]
                / (
                    trajectory_ext[i]["gamma"][j]
                    * c
                    * nhat["R"][j]
                    * (
                        1
                        - np.dot(
                            (
                                trajectory_ext[i]["bx"][j],
                                trajectory_ext[i]["by"][j],
                                trajectory_ext[i]["bz"][j],
                            ),
                            (nhat["nx"][j], nhat["ny"][j], nhat["nz"][j]),
                        )
                    )
                )
            )
    return result